In [1]:
import tensorflow as tf
import scipy, numpy

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

In [2]:
# Configurações para o treinamento
train_data_dir = './data'
batch_size = 32 # 32 imagens por lote
num_epochs = 50 # 50 épocas de treinamento
image_size = (150, 150) # 150x150 pixels
num_classes = 2 # 2 classes: porco e gato

In [3]:
train_datagen = ImageDataGenerator(
    rescale = 1.0/255,
    validation_split = 0.25, # 25% para validação
    shear_range = 0.2, # inclinação
    zoom_range = 0.2, # zoom
    horizontal_flip = True # espelhamento horizontal
)

In [4]:
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size = image_size,
    batch_size = batch_size,
    class_mode = 'binary',
    subset = 'training'
)

Found 750 images belonging to 2 classes.


In [5]:
validation_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size = image_size,
    batch_size = batch_size,
    class_mode = 'binary',
    subset = 'validation'
)

Found 249 images belonging to 2 classes.


In [6]:
model = Sequential() # instanciando o modelo
model.add(Conv2D(32, (3,3), input_shape=(image_size[0], image_size[1], 3), activation='relu')) # camada de convolução
model.add(MaxPooling2D(pool_size=(2,2))) # camada de pooling
model.add(Conv2D(64, (3,3), activation='relu')) # camada de convolução
model.add(MaxPooling2D(pool_size=(2,2))) # camada de pooling
model.add(Flatten()) # camada de achatamento ou vetorização
model.add(Dense(64, activation='relu')) # neurônios
model.add(Dropout(0.5)) # regularização 50% chance de desligar neurônios
model.add(Dense(1, activation='sigmoid')) # camada de saída (classificação binária)

In [7]:
# compilar o modelo
model.compile(
    optimizer = 'adam',
    loss = 'binary_crossentropy', # mais próximo de 0 melhor, 0.002 por exemplo
    metrics = ['accuracy']
)

In [8]:
# treinamento
model.fit(
    train_generator,
    epochs = num_epochs,
    validation_data = validation_generator
)

print('Treinamento concluído!')

# salvar o modelo
model.save('./models/cfModeloArturGG.h5')

Epoch 1/50


24/24 [==============================] - 16s 595ms/step - loss: 0.8763 - accuracy: 0.5013 - val_loss: 0.6928 - val_accuracy: 0.5462
Epoch 2/50
24/24 [==============================] - 10s 434ms/step - loss: 0.6930 - accuracy: 0.5427 - val_loss: 0.6917 - val_accuracy: 0.5020
Epoch 3/50
24/24 [==============================] - 11s 458ms/step - loss: 0.6910 - accuracy: 0.5027 - val_loss: 0.6924 - val_accuracy: 0.5261
Epoch 4/50
24/24 [==============================] - 10s 426ms/step - loss: 0.6894 - accuracy: 0.5387 - val_loss: 0.6890 - val_accuracy: 0.5382
Epoch 5/50
24/24 [==============================] - 10s 416ms/step - loss: 0.6789 - accuracy: 0.5560 - val_loss: 0.6931 - val_accuracy: 0.5181
Epoch 6/50
24/24 [==============================] - 9s 389ms/step - loss: 0.6589 - accuracy: 0.6160 - val_loss: 0.6712 - val_accuracy: 0.5863
Epoch 7/50
24/24 [==============================] - 9s 393ms/step - loss: 0.6526 - accuracy: 0.6147 - val_loss: 0.6742 - val_accuracy: 0.5502

c:\Users\artur\anaconda3\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
